In [23]:
# First, change the folder structure with move_data.ipynb

# Mated adults 



In [24]:
import os

def list_files_from_subfolders(root_dir):
    """
    List files from subdirectories within the 'adults' root directory which contain more than one subdirectory.

    Parameters:
        root_dir (str): The root directory containing the 'adults' directory.

    Returns:
        list of str: Filenames prefixed with their immediate parent directory name from subdirectories that contain more than one subdirectory.
    """
    all_files = []  # List to store filenames
    single_files=[]
    # Check if the root_dir exists and is a directory
    if not os.path.isdir(root_dir):
        print(f"The path {root_dir} is not a valid directory.")
        return all_files

    # Iterate over the items in the 'adults' directory
    for item in os.listdir(root_dir):
        item_path = os.path.join(root_dir, item)
        # Proceed only if the item is a directory
        if os.path.isdir(item_path):
            subdirs = [d for d in os.listdir(item_path) if os.path.isdir(os.path.join(item_path, d))]
            # If the current directory contains more than one subdirectory
            if len(subdirs) > 1:
                # Iterate over each subdirectory
                for subdir in subdirs:
                    subdir_path = os.path.join(item_path, subdir)
                    # Add the file names within this subdirectory to the list
                    files = [os.path.join(item, subdir, f) for f in os.listdir(subdir_path) if os.path.isfile(os.path.join(subdir_path, f))]
                    all_files.extend(files)
            elif len(subdirs)==1:
                for subdir in subdirs:
                    subdir_path = os.path.join(item_path, subdir)
                    # Add the file names within this subdirectory to the list
                    files = [os.path.join(item, subdir, f) for f in os.listdir(subdir_path) if os.path.isfile(os.path.join(subdir_path, f))]
                    single_files.extend(files)

    return all_files, single_files



def list_files_from_folders_with_multiple_files_child(root_dir):
    """
    List files from folders within the 'children' root directory that contain more than one file.

    Parameters:
        root_dir (str): The root directory containing the 'children' directory.

    Returns:
        list of str: Strings representing each file in the folders that contain more than one file.
    """
    folder_files_list = []  # List to store folder/file strings
    single_files_list = []

    # Iterate over the items in the 'children' directory
    for folder_name in os.listdir(root_dir):
        folder_path = os.path.join(root_dir, folder_name)
        # Proceed only if the item is a directory
        if os.path.isdir(folder_path):
            files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
            # Proceed only if the folder contains more than one file
            if len(files) > 1:
                # Create a string for each file
                for file in files:
                    folder_files_list.append(f"{folder_name}/{file}")
            elif len(files) == 1:
                for file in files:
                    single_files_list.append(f"{folder_name}/{file}")
    return folder_files_list,single_files_list





def list_files_from_subfolders_RFW_full(root_dir):
    """
    List files from subdirectories within the 'adults_full' root directory. If a subdirectory contains more than one file,
    these files are added to the `all_files` list. If a subdirectory contains exactly one file, this file is added to
    the `single_files` list.

    Parameters:
        root_dir (str): The root directory containing the 'adults_full' directory.

    Returns:
        tuple: A tuple containing two lists:
               - `all_files` with filenames from directories that contain multiple files,
               - `single_files` with filenames from directories that contain exactly one file.
    """
    all_files = []  # List to store filenames from directories with multiple files
    single_files = []  # List to store filenames from directories with exactly one file

    # Path to the 'adults_full' directory
    adults_full_path = os.path.join(root_dir, 'adults_full')
    # Check if the adults_full_path exists and is a directory
    if not os.path.isdir(adults_full_path):
        print(f"The path {adults_full_path} is not a valid directory.")
        return all_files, single_files

    # Iterate over the items in the 'adults_full' directory
    for item in os.listdir(adults_full_path):
        item_path = os.path.join(adults_full_path, item)
        # Proceed only if the item is a directory
        if os.path.isdir(item_path):
            # Get all files within this directory
            files = [f for f in os.listdir(item_path) if os.path.isfile(os.path.join(item_path, f))]
            # Categorize files based on count in their directory
            if len(files) > 1:
                all_files.extend(os.path.join(item, f) for f in files)
            elif len(files) == 1:
                single_files.append(os.path.join(item, files[0]))

    return all_files, single_files


In [25]:
def extract_unique_identifiers(file_list):
    """
    Extracts a unique identifier from a list of file paths.

    Parameters:
        file_list (list): A list of strings containing file paths.

    Returns:
        list: A list of unique identifiers extracted from the file paths.
    """
    identifiers = []
    for file_path in file_list:
        # Extract the base filename without extension and path
        base_name = os.path.splitext(os.path.basename(file_path))[0]
        identifiers.append(base_name)

    return identifiers


def drop_after_zeros(s):
    """
    Drops everything from the input string after the first occurrence of three consecutive zeros
    and also drops the last underscore before the zeros.

    Parameters:
        s (str): The input string.

    Returns:
        str: The string up to, but not including, the last underscore before the three consecutive zeros.
    """
    # Find the index of the first occurrence of three consecutive zeros
    zero_index = s.find('000')

    # If three consecutive zeros are found
    if zero_index != -1:
        # Find the last underscore before the '000'
        underscore_index = s.rfind('_', 0, zero_index)
        # If an underscore is found, return the substring up to that point
        if underscore_index != -1:
            return s[:underscore_index]
        else:
            return s[:zero_index]
    else:
        return s  # If no '000' sequence is found, return the original string

import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import os
import json
import numpy as np
def load_data(path):
    f = open(path)
    data = json.load(f)
    df = pd.DataFrame.from_dict(data, orient='index', columns=['Age'])
    f.close()

    # removing nans
    nans = df['Age'].isna().sum()
    df = df.dropna()

    describe = df.describe()
    value_counts = df.value_counts()

    # make new column for the identity based on the index
    df['Identity'] = df.index
    df['Identity'] = df.index.str.rsplit('_', 1).str[0]

    df = df.sort_values(by='Age')

    return df, nans, describe, value_counts

import pandas as pd

def drop_number_after_last_underscore(df, column_name):
    """
    Drops the number after the last underscore from the string in the specified column of a DataFrame.

    Parameters:
        df (pd.DataFrame): The DataFrame containing the column to process.
        column_name (str): The name of the column to process.

    Returns:
        pd.DataFrame: The DataFrame with the modified column.
    """
    # Check if the column exists in the DataFrame
    if column_name in df.columns:
        # Split the string by underscore and drop the last part
        df[column_name] = df[column_name].apply(lambda x: '_'.join(x.split('_')[:-1]))
    else:
        raise ValueError(f"The column {column_name} does not exist in the DataFrame.")

    return df


In [26]:
df_YLFW = load_data('/mnt/c/Dokumenter/Dokumenter/UNI/Master/Thesis/GitHub_Repo/Master_Thesis/data/age_estimations/YLFW_output_data_precroped_all_age.json')[0]
df_RFW_african = load_data('/mnt/c/Dokumenter/Dokumenter/UNI/Master/Thesis/GitHub_Repo/Master_Thesis/data/age_estimations/data sendt fra Gabi senest/output_african.json')[0]
df_RFW_asian = load_data('/mnt/c/Dokumenter/Dokumenter/UNI/Master/Thesis/GitHub_Repo/Master_Thesis/data/age_estimations/data sendt fra Gabi senest/output_asian.json')[0]
df_RFW_caucasian = load_data('/mnt/c/Dokumenter/Dokumenter/UNI/Master/Thesis/GitHub_Repo/Master_Thesis/data/age_estimations/data sendt fra Gabi senest/output_caucasian.json')[0]
df_RFW_indian = load_data('/mnt/c/Dokumenter/Dokumenter/UNI/Master/Thesis/GitHub_Repo/Master_Thesis/data/age_estimations/data sendt fra Gabi senest/output_indian.json')[0]
df_RFW = pd.concat([df_RFW_african, df_RFW_asian, df_RFW_caucasian, df_RFW_indian])
df_RFW.Age = df_RFW.Age.astype(int)
df_RFW = df_RFW.sort_values(by='Age', ascending=True)

age_df = pd.concat([df_YLFW,df_RFW])
age_df['image_name'] = age_df.index

# Mated adults

In [27]:

adults_dir = '/mnt/c/Dokumenter/Dokumenter/UNI/Master/Thesis/GitHub_Repo/Master_Thesis/data/raw_full/'
files_list = list_files_from_subfolders_RFW_full(adults_dir)[0]
image_names = extract_unique_identifiers(files_list)


In [28]:
files_list

['African_m.010lz5/m.010lz5_0001.jpg',
 'African_m.010lz5/m.010lz5_0002.jpg',
 'African_m.010lz5/m.010lz5_0003.jpg',
 'African_m.011y5k/m.011y5k_0001.jpg',
 'African_m.011y5k/m.011y5k_0002.jpg',
 'African_m.011y5k/m.011y5k_0003.jpg',
 'African_m.011y5k/m.011y5k_0004.jpg',
 'African_m.01257s/m.01257s_0001.jpg',
 'African_m.01257s/m.01257s_0002.jpg',
 'African_m.01257s/m.01257s_0003.jpg',
 'African_m.0126y2/m.0126y2_0001.jpg',
 'African_m.0126y2/m.0126y2_0002.jpg',
 'African_m.0126y2/m.0126y2_0003.jpg',
 'African_m.0126y2/m.0126y2_0004.jpg',
 'African_m.012f1d/m.012f1d_0001.jpg',
 'African_m.012f1d/m.012f1d_0002.jpg',
 'African_m.012f1d/m.012f1d_0003.jpg',
 'African_m.012gl8/m.012gl8_0001.jpg',
 'African_m.012gl8/m.012gl8_0002.jpg',
 'African_m.012gl8/m.012gl8_0003.jpg',
 'African_m.012gr9/m.012gr9_0001.jpg',
 'African_m.012gr9/m.012gr9_0002.jpg',
 'African_m.012mh_/m.012mh__0002.jpg',
 'African_m.012mh_/m.012mh__0003.jpg',
 'African_m.012mh_/m.012mh__0004.jpg',
 'African_m.012mh_/m.012m

In [29]:

identity_names = []
for i in image_names:
    identity_name = drop_after_zeros(i)

    identity_names.append(identity_name)

DF = pd.DataFrame(
    {'files_list': files_list,
     'image_name': image_names,
     'identity_name': identity_names,
     'enrolled' : 'enrolled'})
DF['ethnicity'] = DF['files_list'].apply(lambda x: x.split('_')[0] if isinstance(x, str) and x.split() else None)
OFIQ_a = pd.read_csv('/mnt/c/Dokumenter/Dokumenter/UNI/Master/Thesis/GitHub_Repo/Master_Thesis/data/OFIQ_results/adults_all_final.csv', sep=';')
OFIQ_c = pd.read_csv('/mnt/c/Dokumenter/Dokumenter/UNI/Master/Thesis/GitHub_Repo/Master_Thesis/data/OFIQ_results/children_all_final.csv', sep=';')
OFIQ_c['image_name']= OFIQ_c['Filename'].apply(lambda x: x.split('/')[-1])
OFIQ_c['image_name']= OFIQ_c['image_name'].apply(lambda x: x.split('.')[0])
OFIQ_a['image_name']=OFIQ_a['Filename'].apply(lambda x: x.split('/')[-1])
OFIQ_a['image_name']=OFIQ_a['image_name'].apply(lambda x: x.split('.jpg')[0])
OFIQ = pd.concat([OFIQ_a,OFIQ_c])

final_adult = pd.merge(DF, age_df, on='image_name', how='left')
final_adult = final_adult.merge(OFIQ[['image_name', 'UnifiedQualityScore.scalar']],
                                on='image_name',
                                how='left')


In [30]:
final_adult

,files_list,image_name,identity_name,enrolled,ethnicity,Age,Identity,UnifiedQualityScore.scalar
0,African_m.010lz5/m.010lz5_0001.jpg,m.010lz5_0001,m.010lz5,enrolled,African,38,m.010lz5,NaN
1,African_m.010lz5/m.010lz5_0002.jpg,m.010lz5_0002,m.010lz5,enrolled,African,29,m.010lz5,NaN
2,African_m.010lz5/m.010lz5_0003.jpg,m.010lz5_0003,m.010lz5,enrolled,African,37,m.010lz5,NaN
3,African_m.011y5k/m.011y5k_0001.jpg,m.011y5k_0001,m.011y5k,enrolled,African,40,m.011y5k,NaN
4,African_m.011y5k/m.011y5k_0002.jpg,m.011y5k_0002,m.011y5k,enrolled,African,43,m.011y5k,NaN
...,...,...,...,...,...,...,...,...
36130,Indian_m.0vshsz9/m.0vshsz9_0002.jpg,m.0vshsz9_0002,m.0vshsz9,enrolled,Indian,36,m.0vshsz9,NaN
36131,Indian_m.0vshsz9/m.0vshsz9_0003.jpg,m.0vshsz9_0003,m.0vshsz9,enrolled,Indian,25,m.0vshsz9,NaN
36132,Indian_m.0_4pw/m.0_4pw_0002.jpg,m.0_4pw_0002,m.0_4pw,enrolled,Indian,64,m.0_4pw,NaN
36133,Indian_m.0_4pw/m.0_4pw_0003.jpg,m.0_4pw_0003,m.0_4pw,enrolled,Indian,37,m.0_4pw,NaN


In [31]:
final_adult.to_csv('mated_adults_image_info.csv', index=False)

In [ ]:
# sammenlign med RFW_full_info_excluding_children.csv , only keep what matched

# Nonmated adults


In [32]:
adults_dir = '/mnt/c/Dokumenter/Dokumenter/UNI/Master/Thesis/GitHub_Repo/Master_Thesis/data/raw_full/'
files_list = list_files_from_subfolders_RFW_full(adults_dir)[1]
image_names = extract_unique_identifiers(files_list)


In [33]:
len(files_list)

376

In [34]:

identity_names = []
for i in image_names:
    identity_name = drop_after_zeros(i)

    identity_names.append(identity_name)

DF = pd.DataFrame(
    {'files_list': files_list,
     'image_name': image_names,
     'identity_name': identity_names,
     'enrolled' : 'non_enrolled'})
DF['ethnicity'] = DF['files_list'].apply(lambda x: x.split('_')[0] if isinstance(x, str) and x.split() else None)
OFIQ_a = pd.read_csv('/mnt/c/Dokumenter/Dokumenter/UNI/Master/Thesis/GitHub_Repo/Master_Thesis/data/OFIQ_results/adults_all_final.csv', sep=';')
OFIQ_c = pd.read_csv('/mnt/c/Dokumenter/Dokumenter/UNI/Master/Thesis/GitHub_Repo/Master_Thesis/data/OFIQ_results/children_all_final.csv', sep=';')
OFIQ_c['image_name']= OFIQ_c['Filename'].apply(lambda x: x.split('/')[-1])
OFIQ_c['image_name']= OFIQ_c['image_name'].apply(lambda x: x.split('.')[0])
OFIQ_a['image_name']=OFIQ_a['Filename'].apply(lambda x: x.split('/')[-1])
OFIQ_a['image_name']=OFIQ_a['image_name'].apply(lambda x: x.split('.jpg')[0])
OFIQ = pd.concat([OFIQ_a,OFIQ_c])

final_adult = pd.merge(DF, age_df, on='image_name', how='left')
final_adult_non = final_adult.merge(OFIQ[['image_name', 'UnifiedQualityScore.scalar']],
                                on='image_name',
                                how='left')
final_adult_non.to_csv('nonmated_adults_image_info.csv', index=False)

In [35]:
final_adult_non

,files_list,image_name,identity_name,enrolled,ethnicity,Age,Identity,UnifiedQualityScore.scalar
0,African_m.01jq08_/m.01jq08__0002.jpg,m.01jq08__0002,m.01jq08_,non_enrolled,African,38,m.01jq08_,NaN
1,African_m.01jr5w/m.01jr5w_0003.jpg,m.01jr5w_0003,m.01jr5w,non_enrolled,African,25,m.01jr5w,NaN
2,African_m.01l_04j/m.01l_04j_0001.jpg,m.01l_04j_0001,m.01l_04j,non_enrolled,African,42,m.01l_04j,NaN
3,African_m.01ng51t/m.01ng51t_0001.jpg,m.01ng51t_0001,m.01ng51t,non_enrolled,African,47,m.01ng51t,NaN
4,African_m.01nk6j7/m.01nk6j7_0003.jpg,m.01nk6j7_0003,m.01nk6j7,non_enrolled,African,40,m.01nk6j7,NaN
...,...,...,...,...,...,...,...,...
372,Indian_m.0jbc3/m.0jbc3_0002.jpg,m.0jbc3_0002,m.0jbc3,non_enrolled,Indian,43,m.0jbc3,NaN
373,Indian_m.0jkv_y7/m.0jkv_y7_0002.jpg,m.0jkv_y7_0002,m.0jkv_y7,non_enrolled,Indian,25,m.0jkv_y7,NaN
374,Indian_m.0j_9xt/m.0j_9xt_0001.jpg,m.0j_9xt_0001,m.0j_9xt,non_enrolled,Indian,49,m.0j_9xt,NaN
375,Indian_m.0k_lgy/m.0k_lgy_0001.jpg,m.0k_lgy_0001,m.0k_lgy,non_enrolled,Indian,34,m.0k_lgy,NaN


# Mated children

In [36]:
child_dir = '/mnt/c/Dokumenter/Dokumenter/UNI/Master/Thesis/GitHub_Repo/Master_Thesis/data/raw_full/YLFW/data_aligned/data_aligned/'
# files_list = list_files_from_subfolders(child_dir)
files_list_child = list_files_from_folders_with_multiple_files_child(child_dir)[0]

image_names = extract_unique_identifiers(files_list_child)

identity_names = []
for i in image_names:
    identity_names.append(drop_after_zeros(i))
DF_child = pd.DataFrame(
    {'files_list': files_list_child,
     'image_name': image_names,
     'enrolled' : 'enrolled'})

DF_child['identity_name'] = DF_child['image_name'].apply(lambda x: '_'.join(x.split('_')[:-1]) if isinstance(x, str) and x.split() else None)
DF_child['ethnicity'] = DF_child['files_list'].apply(lambda x: x.split('_')[0] if isinstance(x, str) and x.split() else None)
final_child = pd.merge(DF_child, age_df, on='image_name', how='left')
final_child_mated = final_child.merge(OFIQ[['image_name', 'UnifiedQualityScore.scalar']],
                                on='image_name',
                                how='left')


In [37]:
final_child_mated

,files_list,image_name,enrolled,identity_name,ethnicity,Age,Identity,UnifiedQualityScore.scalar
0,African_0/African_0_0.png,African_0_0,enrolled,African_0,African,7,African_0,NaN
1,African_0/African_0_1.png,African_0_1,enrolled,African_0,African,18,African_0,34.0
2,African_1/African_1_11.png,African_1_11,enrolled,African_1,African,5,African_1,65.0
3,African_1/African_1_4.png,African_1_4,enrolled,African_1,African,2,African_1,NaN
4,African_10/African_10_1.png,African_10_1,enrolled,African_10,African,5,African_10,NaN
...,...,...,...,...,...,...,...,...
9317,Indian_95/Indian_95_1.png,Indian_95_1,enrolled,Indian_95,Indian,7,Indian_95,NaN
9318,Indian_95/Indian_95_2.png,Indian_95_2,enrolled,Indian_95,Indian,7,Indian_95,37.0
9319,Indian_95/Indian_95_3.png,Indian_95_3,enrolled,Indian_95,Indian,8,Indian_95,42.0
9320,Indian_98/Indian_98_12.png,Indian_98_12,enrolled,Indian_98,Indian,4,Indian_98,NaN


In [39]:
final_child_mated.to_csv('mated_children_image_info.csv', index=False)

# Non mated children

In [38]:
child_dir = '/mnt/c/Dokumenter/Dokumenter/UNI/Master/Thesis/GitHub_Repo/Master_Thesis/data/raw_full/YLFW/data_aligned/data_aligned/'
files_list_child = list_files_from_folders_with_multiple_files_child(child_dir)[1]

image_names = extract_unique_identifiers(files_list_child)

identity_names = []
for i in image_names:
    identity_names.append(drop_after_zeros(i))
DF_child = pd.DataFrame(
    {'files_list': files_list_child,
     'image_name': image_names,
     'enrolled' : 'non_enrolled'})

DF_child['identity_name'] = DF_child['image_name'].apply(lambda x: '_'.join(x.split('_')[:-1]) if isinstance(x, str) and x.split() else None)
DF_child['ethnicity'] = DF_child['files_list'].apply(lambda x: x.split('_')[0] if isinstance(x, str) and x.split() else None)
final_child = pd.merge(DF_child, age_df, on='image_name', how='left')
final_child_non = final_child.merge(OFIQ[['image_name', 'UnifiedQualityScore.scalar']],
                                on='image_name',
                                how='left')
final_child_non.to_csv('nonmated_children_image_info.csv', index=False)

In [40]:
final_child

,files_list,image_name,enrolled,identity_name,ethnicity,Age,Identity
0,African_110/African_110_0.png,African_110_0,non_enrolled,African_110,African,14,African_110
1,African_111/African_111_0.png,African_111_0,non_enrolled,African_111,African,21,African_111
2,African_122/African_122_0.png,African_122_0,non_enrolled,African_122,African,7,African_122
3,African_123/African_123_0.png,African_123_0,non_enrolled,African_123,African,4,African_123
4,African_124/African_124_0.png,African_124_0,non_enrolled,African_124,African,7,African_124
...,...,...,...,...,...,...,...
483,Indian_91/Indian_91_0.png,Indian_91_0,non_enrolled,Indian_91,Indian,15,Indian_91
484,Indian_93/Indian_93_1.png,Indian_93_1,non_enrolled,Indian_93,Indian,10,Indian_93
485,Indian_96/Indian_96_0.png,Indian_96_0,non_enrolled,Indian_96,Indian,8,Indian_96
486,Indian_97/Indian_97_10.png,Indian_97_10,non_enrolled,Indian_97,Indian,18,Indian_97
